In [1]:
import pandas as pd
import os
import numpy as np
from scipy.stats import rankdata
import warnings


from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

import matplotlib.pyplot as plt

from sklearn import model_selection

from sklearn.linear_model import LogisticRegression, RidgeClassifier, Lasso
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import StackingClassifier
import lightgbm as lgb
from xgboost import XGBClassifier

warnings.simplefilter('ignore')

SEED = 2020

def seed_everything(SEED):
    np.random.seed(SEED)
    os.environ['PYTHONHASHSEED'] = str(SEED)

seed_everything(SEED)

In [6]:
# oof_base = pd.read_csv('../input/melanoma-classification-submissions/oof_base.csv')
# test_base = pd.read_csv('../input/melanoma-classification-submissions/test_base.csv')

oof_base = pd.read_csv('../input/melanoma-classification-submissions/Ensemble_oof_base.csv')
test_base = pd.read_csv('../input/melanoma-classification-submissions/Ensemble_test_base.csv')

oof1 = pd.read_csv('../input/melanoma-classification-submissions/oofrank_stacking_0.9556103556665613.csv')
test1 = pd.read_csv('../input/melanoma-classification-submissions/TESTPREDS_RANK_STACKING_0.9556103556665613.csv')

if 'target' in oof_base.columns:
    oof_base = oof_base.drop(['target'],axis=1)
if 'tfrecord' in oof_base.columns:
    oof_base = oof_base.drop(['tfrecord'],axis=1)

train_df = pd.read_csv('../input/melanoma-classification-submissions/train_coded.csv').iloc[:,:-3]
test_df = pd.read_csv('../input/melanoma-classification-submissions/test_coded.csv').iloc[:,:-3]

train_df.head()

,image_name,tfrecord,sex_female,sex_male,age_approx,anatom_site_head/neck,anatom_site_lower extremity,anatom_site_oral/genital,anatom_site_palms/soles,anatom_site_torso,anatom_site_upper extremity,target
0,ISIC_0015719,0,1,0,-0.269395,0,0,0,0,0,1,0
1,ISIC_0052212,6,1,0,0.078659,0,1,0,0,0,0,0
2,ISIC_0068279,0,1,0,-0.269395,1,0,0,0,0,0,0
3,ISIC_0074268,11,1,0,0.426713,0,0,0,0,0,1,0
4,ISIC_0074311,1,1,0,-0.617449,0,1,0,0,0,0,0


In [7]:
oof_base.head()

,image_name,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6
0,ISIC_0052212,0.196309,0.014284,0.191025,0.391115,0.057988,0.167212
1,ISIC_0080817,0.882696,0.189913,0.874424,0.843217,0.770537,0.812890
2,ISIC_0080512,0.165721,0.009689,0.103429,0.391370,0.145995,0.394959
3,ISIC_0078712,0.735265,0.114964,0.704611,0.700642,0.703762,0.350422
4,ISIC_0086349,0.591158,0.044780,0.570354,0.483870,0.516813,0.419047


In [8]:
test_base.head()

,image_name,target_1,target_2,target_3,target_4,target_5,target_6
0,ISIC_0052060,0.176193,0.007751,0.171702,0.243763,0.197932,0.333227
1,ISIC_0052349,0.169651,0.006849,0.125155,0.127541,0.236472,0.197869
2,ISIC_0058510,0.105116,0.004922,0.118434,0.055935,0.060544,0.177336
3,ISIC_0073313,0.143915,0.008863,0.189443,0.196783,0.183630,0.021034
4,ISIC_0073502,0.556200,0.052100,0.529352,0.521823,0.626869,0.432754


In [14]:
oof_base_1 = oof_base.copy()
test_base_1 = test_base.copy()

oof_base_1['pred_2'] = rankdata(oof_base_1['pred_2'])/rankdata(oof_base_1['pred_2']).max()
test_base_1['target_2'] = rankdata(test_base_1['target_2'])/rankdata(test_base_1['target_2']).max()

oof1.pred = rankdata(oof1.pred)/rankdata(oof1.pred).max()
test1.target = rankdata(test1.target)/rankdata(test1.target).max()

oof_base_1['pred_7'] = oof_base_1.merge(oof1[['image_name','pred']],on='image_name',suffixes=('_6','_7'))['pred']
test_base_1['target_7'] = test_base_1.merge(test1,on='image_name',suffixes=('_6','_7'))['target']

oof_target_1 = oof_base_1.merge(train_df[['image_name','target']],on='image_name')[['target']]

for col in oof_base_1.columns[1:]:
    print(f'{col} AUC: {roc_auc_score(oof_target_1,oof_base_1[col])}')
    
order = [3,4,1,5,2,6]

oof_trend = []
test_trend = []

for i in range(len(order)-1):
    oof_trend.append(oof_base_1[f'pred_{order[i]}']-oof_base_1[f'pred_{order[i+1]}'])
    test_trend.append(test_base_1[f'target_{order[i]}']-test_base_1[f'target_{order[i+1]}'])

weight = 1
print("\nWeight: ",weight)
oof_base_1['avg_diff'] = np.mean(oof_trend,axis=0)
test_base_1['avg_diff'] = np.mean(test_trend,axis=0)

print()
print()
test_base_new = pd.DataFrame()
test_base_new['image_name'] = test_base.image_name
oof_base_new = pd.DataFrame()
oof_base_new['image_name'] = oof_base.image_name
aucs = []
for i in range(7):
    pred_col = i+1
    oof_base_new[f'pred_{pred_col}'] = oof_base_1.apply(lambda x: (1+weight*x['avg_diff'])*x[f'pred_{pred_col}'] if x['avg_diff']<0 else (1-weight*x['avg_diff'])*x[f'pred_{pred_col}']+x['avg_diff'],axis=1)
    aucs.append(roc_auc_score(oof_target_1,oof_base_new[f'pred_{pred_col}']))
    print(f"pred_{i+1} New AUC: ",aucs[-1])
    
    test_base_new[f'target_{pred_col}'] = test_base_1.apply(lambda x: (1+weight*x['avg_diff'])*x[f'target_{pred_col}'] if x['avg_diff']<0 else (1-weight*x['avg_diff'])*x[f'target_{pred_col}']+x['avg_diff'],axis=1)
    
# plt.hist(new_sub,bins=100)
# plt.show()
test_base_new.head()

pred_1 AUC: 0.9487946045159296
pred_2 AUC: 0.9437367938054374
pred_3 AUC: 0.9517729781018307
pred_4 AUC: 0.9500662530804962
pred_5 AUC: 0.9470354044605709
pred_6 AUC: 0.9321982091916428
pred_7 AUC: 0.9556103556665613

Weight:  1


pred_1 New AUC:  0.9492002542171515
pred_2 New AUC:  0.9444559536946149
pred_3 New AUC:  0.9516953643640703
pred_4 New AUC:  0.9504170425188746
pred_5 New AUC:  0.9472943759895684
pred_6 New AUC:  0.9362764948671216
pred_7 New AUC:  0.955639246415631


,image_name,target_1,target_2,target_3,target_4,target_5,target_6,target_7
0,ISIC_0052060,0.170501,0.093315,0.166155,0.235888,0.191538,0.322462,0.323828
1,ISIC_0052349,0.167184,0.071518,0.123335,0.125686,0.233033,0.194992,0.111808
2,ISIC_0058510,0.103878,0.023126,0.117039,0.055276,0.059831,0.175247,0.138038
3,ISIC_0073313,0.172749,0.156517,0.216744,0.223837,0.211126,0.054008,0.150358
4,ISIC_0073502,0.564774,0.569222,0.538444,0.531061,0.634078,0.443713,0.543236


In [23]:
oofrank = pd.DataFrame(dict(
    image_name = oof_base_new.image_name,
    pred = oof_base_new.pred_7,
    target = oof_target_1.values[:,0]
))

testrank = pd.DataFrame(dict(
    image_name = test_base_1.image_name,
    target = test_base_1.target_7
))

In [27]:
oofrank.to_csv(f'oofrank_stacking_{aucs[-1]}.csv',index=False)
testrank.to_csv(f'TESTPREDS_RANK_STACKING_{aucs[-1]}.csv',index=False)

In [ ]:
# col = 7
# sub = test_base_new[['image_name']]
# sub['target'] = test_base_new[f'target_{col}']

# sub.to_csv(f'PROBABILITY_CALIB_{aucs[col-1]}.csv',index=False)

# sub.head()

In [ ]:
# plt.hist(sub.target,bins=100)
# plt.show()

In [ ]:
# oof_base = oof_base_new.copy()
# test_base = test_base_new.copy()

oof_base = train_df.merge(oof_base,on='image_name')
test_base = test_df.merge(test_base,on='image_name')

oof_base.head()

In [ ]:
test_base.head()

In [ ]:
# oof_base = oof_base.merge(train_df[['image_name','target','tfrecord']],on='image_name')

y = oof_base[['target']]
tfrecord = oof_base[['tfrecord']]
image_names = oof_base[['image_name']]
X = oof_base.drop(['target','tfrecord','image_name'],axis=1)

X.head()

In [ ]:
X_test = test_base.drop(['image_name'],axis=1)

X_test.columns = X.columns

X_test.head()

In [ ]:
# %%time

# clf1 = KNeighborsClassifier(n_neighbors=3)
# clf2 = RandomForestClassifier(random_state=SEED)
# clf3 = GaussianNB()
# lr = LogisticRegression(random_state=SEED)
# sclf = StackingClassifier(classifiers=[clf1, clf2, clf3],
#                           use_probas=True,
#                           average_probas=False,
#                           meta_classifier=lr)

# print('5-fold cross validation:\n')

# for clf, label in zip([clf1, clf2, clf3, sclf], 
#                       ['KNN', 
#                        'Random Forest', 
#                        'Naive Bayes',
#                        'StackingClassifier']):

#     scores = model_selection.cross_val_score(clf, X, y, cv=5, scoring='roc_auc', n_jobs=-1, verbose=0)
#     print("AUC: %0.6f (+/- %0.6f) [%s]" % (scores.mean(), scores.std(), label))

In [ ]:
FOLDS = 5

skf = KFold(n_splits=FOLDS,shuffle=True,random_state=SEED)

def crossValidate(X=X,y=y,X_test=X_test,tfrecord=tfrecord,image_names=image_names,
                  skf=skf,CLF=LogisticRegression,params=dict(n_jobs= -1),
                  predict_test=False, predict_proba=True):
    print()
    print()
    print("#"*50)
    print("#### CLF: ",CLF)
    
    cvScore = []
    valpreds = []
    valimagenames = []
    valtargets = []

    for fold,(idxT,idxV) in enumerate(skf.split(np.arange(15))):
        print(f"#### FOLD: {fold}")
#         print(f"#### Train idx: {idxT}")
#         print(f"#### Train idx: {idxV}\n")

        idxT, idxV = tfrecord.tfrecord.isin(idxT), tfrecord.tfrecord.isin(idxV)
        
#         print()
#         print("#### Preparing Training and Validation Datasets")
        X_train, y_train = X[idxT], y[idxT]
        X_val, y_val = X[idxV], y[idxV]
        
#         print()
#         print("#### Initializing the Model")
        clf = CLF(**params)
        
#         print()
#         print("#### Training.....")
        clf.fit(X_train,y_train)
        
        
#         print()
#         print("#### Predicting.....")
        if predict_proba:
            print("#### Train AUC: ",roc_auc_score(y_train,clf.predict_proba(X_train)[:,1]))
            pred = clf.predict_proba(X_val)[:,1]
        
        else:
            print("#### Train AUC: ",roc_auc_score(y_train,clf.predict(X_train)))
            pred = clf.predict(X_val)
            
        valpreds.append(pred)
        valimagenames.append(image_names[idxV].values)
        valtargets.append(y_val)

        valauc = roc_auc_score(y_val,pred)
        cvScore.append(valauc)
        print("#### Val AUC: ",valauc)

        print()
    
    valtargets = np.concatenate(valtargets)
#     print(valtargets)
    valpreds = np.concatenate(valpreds)
#     print(valpreds)
    valimagenames = np.concatenate(valimagenames)
    
    auc = roc_auc_score(valtargets,valpreds)
    
    print(f"#### CV: {auc}")
    print(f"#### Mean CV: {np.mean(cvScore)} +/- {np.std(cvScore)}")
    
    oof = pd.DataFrame()
    oof['image_name'] = valimagenames[:,0]
    oof['pred'] = valpreds
    oof['target'] = valtargets[:,0]
    
    sub = None
    if predict_test:
        clf = CLF(**params)
        clf.fit(X,y)
        
        if predict_proba:
            testpreds = clf.predict_proba(X_test)[:,1]
        else:
            testpreds = clf.predict(X_test)

        
        sub = pd.DataFrame(dict(image_name = test_base.image_name, target = testpreds))
    
    print("#"*50)
    
    return cvScore, auc, oof, sub

# if best_auc<auc:
#     print("Score improved!!")
#     best_auc = auc
# else:
#     print("No improvement!!")
#     print("Best AUC: ",best_auc)

In [ ]:
CLF = lgb.LGBMClassifier
params = dict(
    boosting_type='gbdt',
        num_leaves=31,
        max_depth=3,
        learning_rate=0.1,
        n_estimators=110,
        subsample_for_bin=200000,
        objective=None,
        class_weight=None,
        min_split_gain=0.0,
        min_child_weight=0.01,
        min_child_samples=20,
        subsample=0.9,
        subsample_freq=0,
        colsample_bytree=1.0,
        reg_alpha=0.0,
        reg_lambda=0.0,
        random_state=SEED,
        n_jobs=-1,
        silent=True,
        importance_type='split'
)

cvScore, auc, oof, testpreds = crossValidate(CLF=CLF,params=params,predict_test=True)

In [ ]:
??lgb.LGBMClassifier

In [ ]:
base_clfs = [
#     RidgeClassifier,
#     Lasso,
    KNeighborsClassifier,
#     KNeighborsClassifier,
#     KNeighborsClassifier,
#     KNeighborsClassifier,
#     KNeighborsClassifier,

    RandomForestClassifier,
#     RandomForestClassifier,
#     RandomForestClassifier,
#     RandomForestClassifier,
#     RandomForestClassifier,
    
    GaussianNB,
#     GaussianNB,
#     GaussianNB,
#     GaussianNB,
#     GaussianNB,
    
    LogisticRegression,
#     LogisticRegression,
#     LogisticRegression,
#     LogisticRegression,
#     LogisticRegression,

    
    lgb.LGBMClassifier,
#     lgb.LGBMClassifier,
#     lgb.LGBMClassifier,
#     lgb.LGBMClassifier,
#     lgb.LGBMClassifier,
]

base_params = [
#     dict(
#         alpha=0.5,
#         fit_intercept=True,
#         normalize=False,
#         copy_X=True,
#         max_iter=100,
#         tol=0.001,
#         class_weight=None,
#         solver='auto',
#         random_state=SEED
#     ),
    
#     dict(
#         alpha=0.5,
#         fit_intercept=True,
#         normalize=False,
#         precompute=False,
#         copy_X=True,
#         max_iter=1000,
#         tol=0.0001,
#         warm_start=False,
#         positive=False,
#         selection='cyclic',
#         random_state=SEED
#     ),
    
    dict(
        n_neighbors=200,
        weights='uniform',
        algorithm='auto',
        leaf_size=30,
        p=1,
        metric='minkowski',
        metric_params=None,
        n_jobs=-1
    ),
#     dict(
#         n_neighbors=70,
#         weights='uniform',
#         algorithm='auto',
#         leaf_size=30,
#         p=2,
#         metric='minkowski',
#         metric_params=None,
#         n_jobs=-1
#     ),
#     dict(
#         n_neighbors=80,
#         weights='uniform',
#         algorithm='auto',
#         leaf_size=30,
#         p=1,
#         metric='minkowski',
#         metric_params=None,
#         n_jobs=-1
#     ),
#     dict(
#         n_neighbors=80,
#         weights='uniform',
#         algorithm='auto',
#         leaf_size=30,
#         p=2,
#         metric='minkowski',
#         metric_params=None,
#         n_jobs=-1
#     ),
#     dict(
#         n_neighbors=90,
#         weights='uniform',
#         algorithm='auto',
#         leaf_size=30,
#         p=1,
#         metric='minkowski',
#         metric_params=None,
#         n_jobs=-1
#     ),
    
    dict(
        n_estimators=90,
        criterion='gini',
        max_depth=5,
        min_samples_split=3,
        min_samples_leaf=1,
        min_weight_fraction_leaf=0.0,
        max_features='auto',
        max_leaf_nodes=30,
        min_impurity_decrease=0.0,
        min_impurity_split=None,
        bootstrap=True,
        oob_score=False,
        n_jobs=-1,
        random_state=SEED,
        verbose=0,
        warm_start=False,
        class_weight=None,
        ccp_alpha=0.0,
        max_samples=None,
    ),
#     dict(
#         n_estimators=100,
#         criterion='entropy',
#         max_depth=3,
#         min_samples_split=2,
#         min_samples_leaf=1,
#         min_weight_fraction_leaf=0.0,
#         max_features='auto',
#         max_leaf_nodes=None,
#         min_impurity_decrease=0.0,
#         min_impurity_split=None,
#         bootstrap=True,
#         oob_score=False,
#         n_jobs=-1,
#         random_state=SEED,
#         verbose=0,
#         warm_start=False,
#         class_weight=None,
#         ccp_alpha=0.0,
#         max_samples=None,
#     ),
#     dict(
#         n_estimators=100,
#         criterion='gini',
#         max_depth=5,
#         min_samples_split=2,
#         min_samples_leaf=1,
#         min_weight_fraction_leaf=0.0,
#         max_features='auto',
#         max_leaf_nodes=None,
#         min_impurity_decrease=0.0,
#         min_impurity_split=None,
#         bootstrap=True,
#         oob_score=False,
#         n_jobs=-1,
#         random_state=SEED,
#         verbose=0,
#         warm_start=False,
#         class_weight=None,
#         ccp_alpha=0.0,
#         max_samples=None,
#     ),
#     dict(
#         n_estimators=100,
#         criterion='entropy',
#         max_depth=5,
#         min_samples_split=2,
#         min_samples_leaf=1,
#         min_weight_fraction_leaf=0.0,
#         max_features='auto',
#         max_leaf_nodes=None,
#         min_impurity_decrease=0.0,
#         min_impurity_split=None,
#         bootstrap=True,
#         oob_score=False,
#         n_jobs=-1,
#         random_state=SEED,
#         verbose=0,
#         warm_start=False,
#         class_weight=None,
#         ccp_alpha=0.0,
#         max_samples=None,
#     ),
#     dict(
#         n_estimators=100,
#         criterion='gini',
#         max_depth=7,
#         min_samples_split=2,
#         min_samples_leaf=1,
#         min_weight_fraction_leaf=0.0,
#         max_features='auto',
#         max_leaf_nodes=None,
#         min_impurity_decrease=0.0,
#         min_impurity_split=None,
#         bootstrap=True,
#         oob_score=False,
#         n_jobs=-1,
#         random_state=SEED,
#         verbose=0,
#         warm_start=False,
#         class_weight=None,
#         ccp_alpha=0.0,
#         max_samples=None,
#     ),
    
    dict(var_smoothing=1e-1),
#     dict(var_smoothing=1e-8),
#     dict(var_smoothing=1e-7),
#     dict(var_smoothing=1e-6),
#     dict(var_smoothing=1e-5),
    
    dict(
        dual=False,
        tol=0.0001,
        C=1.0,
        fit_intercept=True,
        intercept_scaling=1,
        class_weight=None,
        random_state=SEED,
        solver='lbfgs',
        max_iter=100,
        multi_class='auto',
        verbose=0,
        warm_start=False,
        n_jobs=-1,
        l1_ratio=None,
    ),
#     dict(
#         penalty='l2',
#         dual=False,
#         tol=0.0001,
#         C=0.01,
#         fit_intercept=True,
#         intercept_scaling=1,
#         class_weight=None,
#         random_state=SEED,
#         solver='lbfgs',
#         max_iter=100,
#         multi_class='auto',
#         verbose=0,
#         warm_start=False,
#         n_jobs=-1,
#         l1_ratio=None,
#     ),
#     dict(
#         penalty='l2',
#         dual=False,
#         tol=0.0001,
#         C=0.001,
#         fit_intercept=True,
#         intercept_scaling=1,
#         class_weight=None,
#         random_state=SEED,
#         solver='lbfgs',
#         max_iter=100,
#         multi_class='auto',
#         verbose=0,
#         warm_start=False,
#         n_jobs=-1,
#         l1_ratio=None,
#     ),
#     dict(
#         penalty='l2',
#         dual=False,
#         tol=0.0001,
#         C=0.0001,
#         fit_intercept=True,
#         intercept_scaling=1,
#         class_weight=None,
#         random_state=SEED,
#         solver='lbfgs',
#         max_iter=100,
#         multi_class='auto',
#         verbose=0,
#         warm_start=False,
#         n_jobs=-1,
#         l1_ratio=None,
#     ),
#     dict(
#         penalty='l2',
#         dual=False,
#         tol=0.0001,
#         C=0.00001,
#         fit_intercept=True,
#         intercept_scaling=1,
#         class_weight=None,
#         random_state=SEED,
#         solver='lbfgs',
#         max_iter=100,
#         multi_class='auto',
#         verbose=0,
#         warm_start=False,
#         n_jobs=-1,
#         l1_ratio=None,
#     ),
    
    dict(
        boosting_type='gbdt',
        num_leaves=31,
        max_depth=3,
        learning_rate=0.1,
        n_estimators=110,
        subsample_for_bin=200000,
        objective=None,
        class_weight=None,
        min_split_gain=0.0,
        min_child_weight=0.01,
        min_child_samples=20,
        subsample=0.9,
        subsample_freq=0,
        colsample_bytree=1.0,
        reg_alpha=0.0,
        reg_lambda=0.0,
        random_state=SEED,
        n_jobs=-1,
        silent=True,
        importance_type='split'
    ),
#     dict(
#         boosting_type='dart',
#         num_leaves=31,
#         max_depth=3,
#         learning_rate=0.1,
#         n_estimators=100,
#         subsample_for_bin=200000,
#         objective=None,
#         class_weight=None,
#         min_split_gain=0.0,
#         min_child_weight=0.001,
#         min_child_samples=20,
#         subsample=1.0,
#         subsample_freq=0,
#         colsample_bytree=1.0,
#         reg_alpha=0.0,
#         reg_lambda=0.0,
#         random_state=SEED,
#         n_jobs=-1,
#         silent=True,
#         importance_type='split'
#     ),
#     dict(
#         boosting_type='gbdt',
#         num_leaves=31,
#         max_depth=5,
#         learning_rate=0.1,
#         n_estimators=100,
#         subsample_for_bin=200000,
#         objective=None,
#         class_weight=None,
#         min_split_gain=0.0,
#         min_child_weight=0.001,
#         min_child_samples=20,
#         subsample=1.0,
#         subsample_freq=0,
#         colsample_bytree=1.0,
#         reg_alpha=0.0,
#         reg_lambda=0.0,
#         random_state=SEED,
#         n_jobs=-1,
#         silent=True,
#         importance_type='split'
#     ),
#     dict(
#         boosting_type='dart',
#         num_leaves=31,
#         max_depth=5,
#         learning_rate=0.1,
#         n_estimators=100,
#         subsample_for_bin=200000,
#         objective=None,
#         class_weight=None,
#         min_split_gain=0.0,
#         min_child_weight=0.001,
#         min_child_samples=20,
#         subsample=1.0,
#         subsample_freq=0,
#         colsample_bytree=1.0,
#         reg_alpha=0.0,
#         reg_lambda=0.0,
#         random_state=SEED,
#         n_jobs=-1,
#         silent=True,
#         importance_type='split'
#     ),
#     dict(
#         boosting_type='gbdt',
#         num_leaves=31,
#         max_depth=7,
#         learning_rate=0.1,
#         n_estimators=100,
#         subsample_for_bin=200000,
#         objective=None,
#         class_weight=None,
#         min_split_gain=0.0,
#         min_child_weight=0.001,
#         min_child_samples=20,
#         subsample=1.0,
#         subsample_freq=0,
#         colsample_bytree=1.0,
#         reg_alpha=0.0,
#         reg_lambda=0.0,
#         random_state=SEED,
#         n_jobs=-1,
#         silent=True,
#         importance_type='split'
#     ),
]

In [ ]:
%%time

train_meta = []
test_meta = []
auc_meta = []

for params, clf in zip(base_params,base_clfs):
    
    cvScore, auc, oof, testpreds = crossValidate(CLF=clf,params=params,predict_test=True)
    
    train_meta.append(oof)
    test_meta.append(testpreds)
    auc_meta.append(auc)

print()
print()
print("#### AUCs: ", auc_meta)
print(f"#### Mean AUC: {np.mean(auc_meta)} +/- {np.std(auc_meta)}")
print()
print()

In [ ]:
# 0.945737877503331 +/- 0.006461139964516082

In [ ]:
meta_clf = LogisticRegression
meta_params = dict(
        penalty='l2',
        dual=False,
        tol=0.0001,
        C=0.01,
        fit_intercept=True,
        intercept_scaling=1,
        class_weight=None,
        random_state=SEED,
        solver='lbfgs',
        max_iter=100,
        multi_class='auto',
        verbose=0,
        warm_start=False,
        n_jobs=-1,
        l1_ratio=None,
    )
# meta_clf = XGBClassifier
# meta_params = dict(
#     base_score=0.5,
#     booster='gbtree',
#     colsample_bylevel=0.7,
#     colsample_bynode=0.7,
#     colsample_bytree=0.7,
#     gamma=0,
#     gpu_id=-1,
#     importance_type='gain',
#     interaction_constraints='',
#     learning_rate=0.00300000012,
#     max_delta_step=0,
#     max_depth=5,
#     min_child_weight=1,
#     monotone_constraints='()',
#     n_estimators=100,
#     n_jobs=-1,
#     num_parallel_tree=1,
#     objective='binary:logistic',
#     random_state=SEED,
#     reg_alpha=0,
#     reg_lambda=0,
#     scale_pos_weight=1,
#     subsample=1,
#     tree_method='exact',
#     validate_parameters=1,
#     verbosity=0
# )

X_meta = train_meta[0]
X_test_meta = test_meta[0]

for i, oof in enumerate(train_meta[1:]):
    X_meta = X_meta.merge(oof[['image_name','pred']],on='image_name',suffixes=(f'_{i}',f'_{i+1}'))

for i, testpreds in enumerate(test_meta[1:]):
    X_test_meta = X_test_meta.merge(testpreds,on='image_name',suffixes=(f'_{i}',f'_{i+1}'))
    
X_meta = X_meta.merge(oof_base[['image_name','tfrecord']],on='image_name')

X_ = X_meta.drop(['image_name','target','tfrecord'],axis=1)
y_ = X_meta[['target']]
X_test_ = X_test_meta.drop(['image_name'],axis=1)
tfrecord_ = X_meta[['tfrecord']]
image_names_ = X_meta[['image_name']]

X_test_.columns = X_.columns

cvScore, auc, oof, testpreds = crossValidate(X=X_,y=y_,
                                             X_test=X_test_,
                                             tfrecord=tfrecord_,
                                             image_names=image_names_,
                                             skf=skf,
                                             CLF=meta_clf,
                                             params=meta_params,
                                             predict_test=True,
                                             predict_proba=True
                                            )

In [ ]:
oof.head()

In [ ]:
plt.hist(oof.pred,bins=100)
plt.show()

In [ ]:
testpreds.head()

In [ ]:
plt.hist(testpreds.target,bins=100)
plt.show()

In [ ]:
testpreds.to_csv(f'TESTPREDS_STACKING_{auc}.csv',index=False)
oof.to_csv(f'oof_stacking_{auc}.csv',index=False)